In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
import pickle

In [ ]:
import numpy as np
from scipy.stats import poisson, norm

In [ ]:
def reflected_random_walk(demand_seq,transfer_seq, flow=False):
    state = np.zeros(len(demand_seq)+1)
    for i,(d,t) in enumerate(zip(demand_seq,transfer_seq)):
        if flow:
            t = min(state[i]+d,t)
        else:
            t = min(state[i],t)
        state[i+1] = max(state[i]+d-t,0)
        
    return state[1:]

In [ ]:
n = int(1e6)

mu_demand = 30
#mu_transfer = mu_demand*1.03
mu_drain = mu_demand*1.03

mu_transfer = mu_drain+(mu_drain-mu_demand)*1


np.random.seed(100)

demand_seq = np.random.poisson(mu_demand,n)
transfer_seq = np.random.poisson(mu_transfer,n)
drain_seq = np.random.poisson(mu_drain,n)

#demand_seq = np.ones(n)*mu_demand
#transfer_seq = np.ones(n)*mu_transfer
#drain_seq = np.ones(n)*mu_drain

#demand_seq = norm.rvs(mu_demand,np.sqrt(mu_demand),n)
#supply_seq = norm.rvs(mu_supply,np.sqrt(mu_supply),n)

In [ ]:
percentile = 1.645
#percentile = 0
percentile = 1.96
#percentile = 2.33
percentile = 3.1
#percentile = 1
#percentile = 7
slack_capacity = mu_transfer-mu_drain
std = np.sqrt(mu_drain+mu_transfer)
time_horizon = (percentile*std)**2/(2*slack_capacity)**2
offset = time_horizon*(-slack_capacity) + percentile*std*np.sqrt(time_horizon)
time_horizon = int(np.ceil(time_horizon))
offset = int(np.ceil(offset))

In [ ]:
x = np.arange(0,20,1)
plt.plot(x,2*x/(x+5))

In [ ]:
offset

In [ ]:
percentile = 3.0
slack_capacity = mu_transfer-mu_drain
std = np.sqrt(mu_drain+mu_transfer)

beta = (1/4)*(percentile**2)*(std/slack_capacity)# + slack_capacity/std

In [ ]:
std/mu_demand

In [ ]:
beta

In [ ]:
mu_drain

In [ ]:
def compute_scaling_ratio(mu_drain,mu_demand,drift_sd,init_state):
    drain_time = init_state/(mu_drain-mu_demand)
    accum_std = drift_sd*np.sqrt(drain_time)
    ratio = accum_std/init_state
    
    return ratio

In [ ]:
c1 = 1
c2 = 3

In [ ]:
def simulate_maxweight(arrival_sequence,transfer_sequence,drain_sequence,c1,c2,init_state,eps):
    arrival_buffer = np.zeros(len(arrival_sequence)+1)
    drain_buffer = np.zeros(len(arrival_sequence)+1)
    #drain_buffer[0] = init_state
    arrival_buffer[0] = init_state[0]
    drain_buffer[0] = init_state[1]
    
    n_idles = 0
    reached = False
    for i,(a,t,d) in enumerate(zip(arrival_sequence,transfer_sequence,drain_sequence)):
        t = min(t,arrival_buffer[i])
        d = min(d,drain_buffer[i])
        if drain_buffer[i]*c2/(drain_buffer[i]+eps) >= c1:
            t = 0
            n_idles += 1
            
        elif not reached:
            reached = True
            thres = 1-drain_buffer[i]*1./init_state
        
        arrival_buffer[i+1] = max(0,arrival_buffer[i]+a-t)
        drain_buffer[i+1] = max(0,drain_buffer[i]+t-d)
        
    #print(n_idles,idling_benefit,idling_benefit_2)
    
    print(n_idles/len(arrival_sequence))
    return arrival_buffer,drain_buffer,thres

In [ ]:
in_state = 7000
f,ax = plt.subplots(3,1,figsize=(16,8))
duration = 10000
plot_range = range(duration)
cumul=False

np.random.seed(5)

dem_seq = np.random.poisson(mu_demand,duration)
trans_seq = np.random.poisson(mu_transfer,duration)
dr_seq = np.random.poisson(mu_drain,duration)

safety_stocks_range = np.arange(0,50,1)

costs = {}

for safety_mult in safety_stocks_range:
    
    safety_stock = 2*mu_drain+std*safety_mult
    
    eps = safety_stock*(c2/c1-1)
    init_state = [in_state,safety_stock]
    
    arrival_buffer,drain_buffer,mw_thres = simulate_maxweight(dem_seq[:duration],
                                                                  trans_seq[:duration],
                                                                  dr_seq[:duration],
                                                                  c1,c2,init_state=init_state,
                                                              eps=eps)
    max_weight_cost = (sum(arrival_buffer*c1)+sum(drain_buffer*c2))
    
    costs[safety_mult] = max_weight_cost
        #max_weight_transfer_ratio = maxweight_transfer_ratio(mu_demand,mu_transfer,mu_drain,c1,c2)

    if cumul:
        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range])
        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        #pass
        ax[1].plot((drain_buffer*c2)[plot_range])
        ax[0].plot((arrival_buffer*c1)[plot_range])
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))

In [ ]:
safety_stocks = np.array(list(costs.keys()))
cost_values = np.array(list(costs.values()))
cost_values = cost_values/min(cost_values)

In [ ]:
indices = np.where(cost_values < 1.3)[0]
plt.plot(safety_stocks[indices],cost_values[indices])
plt.vlines(beta,min(cost_values[indices]),max(cost_values[indices]))

In [ ]:
plt.plot(drain_buffer[300:1600])

In [ ]:
x = np.arange(1,1000,1)

In [ ]:
300/310

In [ ]:
beta = (1/4)*(percentile**2)*(std/slack_capacity) + slack_capacity/std

In [ ]:
x = np.arange(0,20,1)
plt.plot(x,2*x**1/(x**1+10),"o")

In [ ]:
x = np.arange(0,20,1)
plt.plot(x[:-1],np.diff(x**2/(x**2+10)),"o")
#plt.yscale("log",basey=10)

In [ ]:
mw_thres

In [ ]:
scale = 1

flow=False
deterministic = False
cumul = False

f,ax = plt.subplots(3,1,figsize=(16,8))

init_state = int(7e4*scale)
duration = int(110000*scale)#len(demand_seq)
#uration = int(140000*scale)
plot_range = range(duration)
#plot_range = range(duration-200,duration)

thresholds = []
x_incr = 0.2
x_transfer = np.arange(0,1.+x_incr,x_incr)
x_transfer = [0.65]

if deterministic:
    dem_seq = np.ones(duration)*mu_demand
    trans_seq = np.ones(duration)*mu_transfer
    dr_seq = np.ones(duration)*mu_drain
    drift_sd = 0
    #if flow:
     #   drift_sd = -1
    #drift_sd = np.sqrt(2*mu_drain)
else:
    np.random.seed(init_state)
    dem_seq = np.random.poisson(mu_demand,duration)
    trans_seq = np.random.poisson(mu_transfer,duration)
    dr_seq = np.random.poisson(mu_drain,duration)
    drift_sd = np.sqrt(2*mu_drain)
    #if flow:
    #drift_sd = 0
    #drift_sd = 1

scaling_ratio = compute_scaling_ratio(mu_drain,mu_demand,drift_sd,init_state)
    
#print(len(dem_seq))
for x_t in x_transfer:

    thres = init_state-x_t*init_state
    thresholds.append(thres)
    arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk(dem_seq[:duration],
                                                                      trans_seq[:duration],
                                                                      dr_seq[:duration],thres,init_state=init_state,
                                                                        flow=flow)

    if cumul:
        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=x_t)
        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        #pass
        ax[1].plot((drain_buffer*c2)[plot_range],label=x_t)
        ax[0].plot((arrival_buffer*c1)[plot_range])
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))
f.suptitle(f'{np.round(scaling_ratio,3)}',fontsize=30)

In [ ]:
costs = {}

In [ ]:
scale_list = [0.03,0.07,0.1,0.3,1,2]
#scale_list = [0.03,0.07,0.1,0.3]
#scale_list = [0.1]
closed_loop = True
flow = False
deterministic = False
plot = False


for scale in scale_list:
    print(scale)
    cost = []
    drain_cost = []
    cost_1 = []
    hedging = np.arange(100,5000,250)
    init_state = int(7e4*scale)
    #hedging = np.arange(1,7)
    j = 1
    if plot:
        f,ax = plt.subplots(3,1,figsize=(16,8))
    #plot_range = range(4000,5000)

    opt_transfer_ratio = optimal_transfer_ratio(mu_demand,mu_transfer,mu_drain,c1,c2)

    duration = int(110000*scale)#len(demand_seq)
    duration = int(140000*scale)
    n_repeat = max(1,int(1/scale))*3
    plot_range = range(duration)
    #plot_range = range(duration-200,duration)
    cumul = False
    thresholds = []
    #x_incr = 2000*scale
    #x_transfer = np.arange(0,init_state,x_incr)
    x_incr = 0.05
    x_transfer = np.arange(0,1.+x_incr,x_incr)
    #for h in reversed(hedging):
    
    if plot:
        n_repeat = 1

    sample_costs = []
    for n in range(1,n_repeat+1):
        cost = []
        if deterministic:
            dem_seq = np.ones(duration)*mu_demand
            trans_seq = np.ones(duration)*mu_transfer
            dr_seq = np.ones(duration)*mu_drain
            drift_sd = 0
            #if flow:
             #   drift_sd = -1
            #drift_sd = np.sqrt(2*mu_drain)
        else:
            np.random.seed(init_state*n)
            dem_seq = np.random.poisson(mu_demand,duration)
            trans_seq = np.random.poisson(mu_transfer,duration)
            dr_seq = np.random.poisson(mu_drain,duration)
            drift_sd = np.sqrt(2*mu_drain)
            #if flow:
            #drift_sd = 0
            #drift_sd = 1
        
        #print(len(dem_seq))
        for x_t in x_transfer:
            #print(x_t)
            #thres = mu_drain+h*np.sqrt(mu_drain)
            thres = init_state-x_t*init_state
            thresholds.append(thres)
            #print(thres)
            #thres = h*10
            
            n_idles = int(x_t*init_state/mu_drain)
            #print(x_t,n_idles)
            if closed_loop:
                arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk(dem_seq[:duration],
                                                                              trans_seq[:duration],
                                                                              dr_seq[:duration],thres,init_state=init_state,
                                                                                flow=flow)
            else:
                arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk_open_loop(
                    dem_seq[:duration],
                    trans_seq[:duration],
                    dr_seq[:duration],
                    n_idles,
                    init_state=init_state,
                    flow=flow)
            
            
            a_zeros = np.where(np.array(arrival_buffer) < mu_transfer)[0]
            if len(a_zeros) < 3:
                final_a_ind = len(arrival_buffer)
            else:
                final_a_ind = a_zeros[2]
                
            
            
            d_zeros = np.where(np.array(drain_buffer) < mu_drain)[0]
            #if len(d_zeros) < 2:
             #   final_d_ind = len(drain_buffer)
            #else:
             #   final_d_ind = d_zeros[1]
            final_d_ind = d_zeros[0]
                
            #print(final_d_ind)
            #print()
            #print(x_t)
            #print(final_a_ind,sum(arrival_buffer[:final_a_ind]*c1)/sum(arrival_buffer*c1))
            #print(final_d_ind,sum(drain_buffer[:final_d_ind])/sum(drain_buffer))
            
            #cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
            clean_cost = sum(arrival_buffer[:final_a_ind]*c1)+sum(drain_buffer[:final_d_ind]*c2)
            total_cost = sum(arrival_buffer*c1)+sum(drain_buffer*c2)
            #print(clean_cost/total_cost)
            #print(sum(arrival_buffer[:final_a_ind]*c1)+sum(drain_buffer[:final_d_ind]*c2)))
            
            
            cost.append(clean_cost)
            #drain_cost.append(init_state**2*c1/(2*(mu_transfer*zeta-mu_demand)))
            #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
            if plot:
                    if cumul:
                        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=x_t)
                        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
                        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
                    else:
                        #pass
                        ax[1].plot((drain_buffer*c2)[plot_range],label=x_t)
                        ax[0].plot((arrival_buffer*c1)[plot_range])
                        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
                #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))

        cost_norm = 0.5*c2*init_state**2/(mu_drain-mu_demand)



        #drift_sd = 0
        scaling_ratio = compute_scaling_ratio(mu_drain,mu_demand,drift_sd,init_state)
        sample_costs.append(np.array(cost))
        
        #costs[(scaling_ratio,scale,init_state)] = cost

        arrival_buffer,drain_buffer,mw_thres = simulate_maxweight(dem_seq[:duration],
                                                              trans_seq[:duration],
                                                              dr_seq[:duration],
                                                              c1,c2,init_state=init_state)
        max_weight_cost = (sum(arrival_buffer*c1)+sum(drain_buffer*c2))
        max_weight_transfer_ratio = maxweight_transfer_ratio(mu_demand,mu_transfer,mu_drain,c1,c2)

        remainder_ratio = 1-opt_transfer_ratio

        arrival_buffer,drain_buffer,zeta = simulate_adjustable_threshold(dem_seq[:duration],
                                                                         trans_seq[:duration],
                                                                         dr_seq[:duration],remainder_ratio,init_state=init_state)
        adjust_thres_cost = (sum(arrival_buffer*c1)+sum(drain_buffer*c2))

        #ax[1].plot((drain_buffer*c2)[plot_range],label="hello")
        #ax[0].plot((arrival_buffer*c1)[plot_range])
        #ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])

        #thres = 2e6
        #thres = h*10
        #arrival_buffer,drain_buffer,_ = simulate_reflected_random_walk(demand_seq[:duration],transfer_seq[:duration],drain_seq[:duration],thres,init_state=init_state)
        #no_idling_cost = sum(arrival_buffer*c1)+sum(drain_buffer*c2)
        #cost.append(sum(arrival_buffer*c1))
        #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
        #if cumul:
         #   ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label="e")
          #  ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range],label="e")
           # ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
        #else:
         #   ax[1].plot((drain_buffer*c2)[plot_range],label="e")
          #  ax[0].plot((arrival_buffer*c1)[plot_range])
           # ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])

        #f.legend()
    costs[(scaling_ratio,scale,init_state)] = np.mean(np.vstack(sample_costs),axis=0)

In [ ]:
scale_list = [0.03,0.07,0.1,0.3,1,2]
#scale_list = [0.03,0.07,0.1,0.3]
#scale_list = [0.1]
closed_loop = True
flow = False
deterministic = True
plot = False


for scale in scale_list:
    print(scale)
    cost = []
    drain_cost = []
    cost_1 = []
    hedging = np.arange(100,5000,250)
    init_state = int(7e4*scale)
    #hedging = np.arange(1,7)
    j = 1
    if plot:
        f,ax = plt.subplots(3,1,figsize=(16,8))
    #plot_range = range(4000,5000)

    opt_transfer_ratio = optimal_transfer_ratio(mu_demand,mu_transfer,mu_drain,c1,c2)

    duration = int(110000*scale)#len(demand_seq)
    duration = int(140000*scale)
    n_repeat = max(1,int(1/scale))*3
    plot_range = range(duration)
    #plot_range = range(duration-200,duration)
    cumul = False
    thresholds = []
    #x_incr = 2000*scale
    #x_transfer = np.arange(0,init_state,x_incr)
    x_incr = 0.05
    x_transfer = np.arange(0,1.+x_incr,x_incr)
    #for h in reversed(hedging):
    
    if plot:
        n_repeat = 1

    sample_costs = []
    for n in range(1,n_repeat+1):
        cost = []
        if deterministic:
            dem_seq = np.ones(duration)*mu_demand
            trans_seq = np.ones(duration)*mu_transfer
            dr_seq = np.ones(duration)*mu_drain
            drift_sd = 0
            #if flow:
             #   drift_sd = -1
            #drift_sd = np.sqrt(2*mu_drain)
        else:
            np.random.seed(init_state*n)
            dem_seq = np.random.poisson(mu_demand,duration)
            trans_seq = np.random.poisson(mu_transfer,duration)
            dr_seq = np.random.poisson(mu_drain,duration)
            drift_sd = np.sqrt(2*mu_drain)
            #if flow:
            #drift_sd = 0
            #drift_sd = 1
        
        #print(len(dem_seq))
        for x_t in x_transfer:
            #print(x_t)
            #thres = mu_drain+h*np.sqrt(mu_drain)
            thres = init_state-x_t*init_state
            thresholds.append(thres)
            #print(thres)
            #thres = h*10
            
            n_idles = int(x_t*init_state/mu_drain)
            #print(x_t,n_idles)
            if closed_loop:
                arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk(dem_seq[:duration],
                                                                              trans_seq[:duration],
                                                                              dr_seq[:duration],thres,init_state=init_state,
                                                                                flow=flow)
            else:
                arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk_open_loop(
                    dem_seq[:duration],
                    trans_seq[:duration],
                    dr_seq[:duration],
                    n_idles,
                    init_state=init_state,
                    flow=flow)
            
            
            a_zeros = np.where(np.array(arrival_buffer) < mu_transfer)[0]
            if len(a_zeros) < 3:
                final_a_ind = len(arrival_buffer)
            else:
                final_a_ind = a_zeros[2]
                
            
            
            d_zeros = np.where(np.array(drain_buffer) < mu_drain)[0]
            #if len(d_zeros) < 2:
             #   final_d_ind = len(drain_buffer)
            #else:
             #   final_d_ind = d_zeros[1]
            final_d_ind = d_zeros[0]
                
            #print(final_d_ind)
            #print()
            #print(x_t)
            #print(final_a_ind,sum(arrival_buffer[:final_a_ind]*c1)/sum(arrival_buffer*c1))
            #print(final_d_ind,sum(drain_buffer[:final_d_ind])/sum(drain_buffer))
            
            #cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
            clean_cost = sum(arrival_buffer[:final_a_ind]*c1)+sum(drain_buffer[:final_d_ind]*c2)
            total_cost = sum(arrival_buffer*c1)+sum(drain_buffer*c2)
            #print(clean_cost/total_cost)
            #print(sum(arrival_buffer[:final_a_ind]*c1)+sum(drain_buffer[:final_d_ind]*c2)))
            
            
            cost.append(clean_cost)
            #drain_cost.append(init_state**2*c1/(2*(mu_transfer*zeta-mu_demand)))
            #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
            if plot:
                if cumul:
                    ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=x_t)
                    ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
                    ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
                else:
                    #pass
                    ax[1].plot((drain_buffer*c2)[plot_range],label=x_t)
                    ax[0].plot((arrival_buffer*c1)[plot_range])
                    ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
            #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))

        cost_norm = 0.5*c2*init_state**2/(mu_drain-mu_demand)



        #drift_sd = 0
        scaling_ratio = compute_scaling_ratio(mu_drain,mu_demand,drift_sd,init_state)
        sample_costs.append(np.array(cost))
        
        #costs[(scaling_ratio,scale,init_state)] = cost

        arrival_buffer,drain_buffer,mw_thres = simulate_maxweight(dem_seq[:duration],
                                                              trans_seq[:duration],
                                                              dr_seq[:duration],
                                                              c1,c2,init_state=init_state)
        max_weight_cost = (sum(arrival_buffer*c1)+sum(drain_buffer*c2))
        max_weight_transfer_ratio = maxweight_transfer_ratio(mu_demand,mu_transfer,mu_drain,c1,c2)

        remainder_ratio = 1-opt_transfer_ratio

        arrival_buffer,drain_buffer,zeta = simulate_adjustable_threshold(dem_seq[:duration],
                                                                         trans_seq[:duration],
                                                                         dr_seq[:duration],remainder_ratio,init_state=init_state)
        adjust_thres_cost = (sum(arrival_buffer*c1)+sum(drain_buffer*c2))

        #ax[1].plot((drain_buffer*c2)[plot_range],label="hello")
        #ax[0].plot((arrival_buffer*c1)[plot_range])
        #ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])

        #thres = 2e6
        #thres = h*10
        #arrival_buffer,drain_buffer,_ = simulate_reflected_random_walk(demand_seq[:duration],transfer_seq[:duration],drain_seq[:duration],thres,init_state=init_state)
        #no_idling_cost = sum(arrival_buffer*c1)+sum(drain_buffer*c2)
        #cost.append(sum(arrival_buffer*c1))
        #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
        #if cumul:
         #   ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label="e")
          #  ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range],label="e")
           # ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
        #else:
         #   ax[1].plot((drain_buffer*c2)[plot_range],label="e")
          #  ax[0].plot((arrival_buffer*c1)[plot_range])
           # ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])

        #f.legend()
    costs[(scaling_ratio,scale,init_state)] = np.mean(np.vstack(sample_costs),axis=0)

In [ ]:
mu_drain

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(x_transfer,np.array(cost)/cost_norm,".")

plt.vlines(optimal_transfer_ratio,min(np.array(cost)/cost_norm)*0.9,1,"r")

plt.vlines(max_weight_transfer_ratio,min(np.array(cost)/cost_norm)*0.9,1,"k")
plt.plot(mw_thres,max_weight_cost/cost_norm,"ko",markersize=10)
plt.hlines(adjust_thres_cost/cost_norm,0,1,"b")

In [ ]:
#plt.plot(hedging,[c for c in drain_cost[::-1]])
plt.plot(x_transfer,np.array(cost)/cost_norm,".")
plt.vlines(optimal_ratio,min(np.array(cost)/cost_norm)*0.9,1,"r")

In [ ]:
with open("deterministic_flow.pkl","wb") as f:
    pickle.dump(costs,f)

In [ ]:
with open("cost_profiles.pkl","rb") as f:
    costs = pickle.load(f)

In [ ]:
costs.keys()

In [ ]:
#x_transfer = np.arange(0,1,0.02)

plt.figure(figsize=(12,8))
for (ratio,scale,init_state) in sorted(costs,key=lambda x: (x[0],-x[1])):
    cost = costs[(ratio,scale,init_state)]
    x_transfer = np.arange(0,1+1./(len(cost)-1),1./(len(cost)-1))
    
    #if scale != 2:
     #   continue
        
    #cost_norm = costs[(0.,init_state)][0]
    #cost_norm = cost[0]
    cost_norm = 0.5*c2*init_state**2/(mu_drain-mu_demand)
    
    if ratio == 0:
        #continue
        plt.plot(x_transfer,np.array(cost)/cost_norm,"-",label="{:.2}, {}".format(ratio,scale))
    else:
        #continue
        plt.plot(x_transfer,np.array(cost)/cost_norm,".",label="{:.2}, {}".format(ratio,scale))

plt.vlines(optimal_ratio,min(np.array(cost)/cost_norm)*0.8,1,"r")
plt.ylim(top=1.1)    
plt.ylabel("Relative accumulated cost")
plt.xlabel("Transfer ratio")
plt.legend()

In [ ]:
#x_transfer = np.arange(0,1,0.02)

plt.figure(figsize=(12,8))
for (ratio,scale,init_state) in sorted(costs,key=lambda x: x):
    cost = costs[(ratio,scale,init_state)]
    x_transfer = np.arange(0,1+1./(len(cost)-1),1./(len(cost)-1))
    
    #if scale != 2:
     #   continue
        
    #cost_norm = costs[(0.,init_state)][0]
    #cost_norm = cost[0]
    cost_norm = 0.5*c2*init_state**2/(mu_drain-mu_demand)
    
    if ratio == 0:
        #continue
        plt.plot(x_transfer,np.array(cost)/cost_norm,"-",label="{:.2}, {}".format(ratio,scale))
    else:
        #continue
        plt.plot(x_transfer,np.array(cost)/cost_norm,".",label="{:.2}, {}".format(ratio,scale))

plt.vlines(optimal_ratio,min(np.array(cost)/cost_norm)*0.8,1,"r")
plt.ylim(top=1.1)    
plt.ylabel("Relative accumulated cost")
plt.xlabel("Transfer ratio")
plt.legend()

In [ ]:
#x_transfer = np.arange(0,1,0.02)

plt.figure(figsize=(12,8))
for (ratio,scale,init_state) in sorted(costs,key=lambda x: x):
    cost = costs[(ratio,scale,init_state)]
    x_transfer = np.arange(0,1+1./(len(cost)-1),1./(len(cost)-1))
    
    #if scale != 2:
     #   continue
        
    #cost_norm = costs[(0.,init_state)][0]
    #cost_norm = cost[0]
    cost_norm = 0.5*c2*init_state**2/(mu_drain-mu_demand)
    
    if ratio == 0:
        #continue
        plt.plot(x_transfer,np.array(cost)/cost_norm,"-",label="{:.2}, {}".format(ratio,scale))
    else:
        #continue
        plt.plot(x_transfer,np.array(cost)/cost_norm,".",label="{:.2}, {}".format(ratio,scale))

plt.vlines(optimal_ratio,min(np.array(cost)/cost_norm)*0.8,1,"r")
plt.ylim(top=1.1)    
plt.ylabel("Relative accumulated cost")
plt.xlabel("Transfer ratio")
plt.legend()

In [ ]:
plt.figure(figsize=(12,8))
for (ratio,init_state) in sorted(costs,key=lambda x: x):
    cost = costs[(ratio,init_state)]
    #if init_state != 21000:
     #   continue
        
    #cost_norm = costs[(0.,init_state)][0]
    cost_norm = 0.5*c2*init_state**2/(mu_drain-mu_demand)
    
    if ratio == 0:
        #continue
        plt.plot(x_transfer,np.array(cost)/cost_norm,"-",label="{:.2}, {}".format(ratio,init_state))
    else:
        #continue
        plt.plot(x_transfer,np.array(cost)/cost_norm,".",label="{:.2}, {}".format(ratio,init_state))

plt.vlines(optimal_ratio,min(np.array(cost)/cost_norm)*0.8,1,"r")
plt.ylim(top=1.1)    
plt.ylabel("Relative accumulated cost")
plt.xlabel("Transfer ratio")
plt.legend()

In [ ]:
#with open("cost_profiles.pkl","wb") as f:
 #   pickle.dump(costs,f)

In [ ]:
costs.keys()

In [ ]:
errors = {}
optimal_cost = 0.5*c2*140000**2/(mu_drain-mu_demand)
cost_det = np.array(costs[(0.0, 2, 140000)])/optimal_cost

for (ratio,scale,init_state),cost in costs.items():
        
    if ratio != 0:
        cost_norm = 0.5*c2*init_state**2/(mu_drain-mu_demand)
        cost = np.array(cost)/cost_norm
        dif = (cost-cost_det)
        error = np.sqrt(np.mean((dif)**2))
        #error = np.mean((dif)**2)
        #error = np.mean(np.abs(dif))
        errors[ratio] = error

In [ ]:
errors

In [ ]:
plt.plot(errors.keys(),errors.values(),".")

In [ ]:
#plt.plot(hedging,[c for c in drain_cost[::-1]])
plt.plot(x_transfer/init_state,cost/cost[0],".")

In [ ]:
#plt.plot(hedging,[c for c in drain_cost[::-1]])
plt.plot(x_transfer/init_state,[c for c in cost[::-1]],".")

In [ ]:
cost = []
hedging = np.arange(0,60,5)
init_state = 0
#hedging = np.arange(1,7)
j = 1
f,ax = plt.subplots(3,1,figsize=(16,8))
#plot_range = range(4000,5000)

duration = 55000
plot_range = range(0,50000)
#plot_range = range(0,300)
cumul =True
thresholds= []
for h in reversed(hedging):
    thres = mu_drain+h*np.sqrt(mu_drain)
    thresholds.append(thres)
    #thres = h*10
    arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk(demand_seq[:duration],transfer_seq[:duration],drain_seq[:duration],thres,init_state=init_state)
    cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
    #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
    if cumul:
        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=h)
        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        ax[1].plot((drain_buffer*c2)[plot_range],label=h)
        ax[0].plot((arrival_buffer*c1)[plot_range])
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))
f.legend()

In [ ]:
cost

In [ ]:
min_cost = min(cost)
plt.plot(hedging,[c/no_idling_cost for c in cost[::-1]])
plt.plot(hedging,[c/no_idling_cost for c in cost[::-1]],".")

In [ ]:
min_cost = min(cost)
plt.plot(hedging,[c/min_cost for c in cost[::-1]])
plt.plot(hedging,[c/min_cost for c in cost[::-1]],".")

# drain resource is the main bottleneck

In [ ]:
n = int(1e6)

mu_demand = 65
mu_drain = mu_demand*1.03
mu_transfer = mu_drain*1.03

np.random.seed(5)

demand_seq = np.random.poisson(mu_demand,n)
transfer_seq = np.random.poisson(mu_transfer,n)
drain_seq = np.random.poisson(mu_drain,n)

#demand_seq = norm.rvs(mu_demand,np.sqrt(mu_demand),n)
#supply_seq = norm.rvs(mu_supply,np.sqrt(mu_supply),n)

In [ ]:
cost = []
rates = []
hedging = np.arange(0,60,5)
init_state = 1e4
#hedging = np.arange(1,7)
j = 1
f,ax = plt.subplots(3,1,figsize=(16,8))
#plot_range = range(4000,5000)

duration = 10000
plot_range = range(0,10000)
#plot_range = range(0,300)
cumul =False
for h in reversed(hedging):
    thres = mu_drain+h*np.sqrt(mu_drain)
    #thres = h*10
    arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk(demand_seq[:duration],transfer_seq[:duration],drain_seq[:duration],thres,init_state=init_state)
    cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
    rates.append(zeta*mu_transfer)
    #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
    if cumul:
        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=h)
        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        ax[1].plot((drain_buffer*c2)[plot_range],label=h)
        ax[0].plot((arrival_buffer*c1)[plot_range])
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))
f.legend()

In [ ]:
mu_transfer

In [ ]:
rates

In [ ]:
mu_drain

In [ ]:
cost = []
hedging = np.arange(0,60,5)
init_state = 0
#hedging = np.arange(1,7)
j = 1
f,ax = plt.subplots(3,1,figsize=(16,8))
#plot_range = range(4000,5000)

duration = 100000
plot_range = range(0,10000)
#plot_range = range(0,200)
cumul =False
for h in reversed(hedging):
    thres = mu_drain+h*np.sqrt(mu_drain)
    #thres = h*10
    arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk(demand_seq[:duration],transfer_seq[:duration],drain_seq[:duration],thres,init_state=init_state)
    cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
    #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
    if cumul:
        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=h)
        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        ax[1].plot((drain_buffer*c2)[plot_range],label=h)
        ax[0].plot((arrival_buffer*c1)[plot_range])
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))
f.legend()

In [ ]:
min_cost = min(cost)
plt.plot(hedging,[c/min_cost for c in cost[::-1]])
plt.plot(hedging,[c/min_cost for c in cost[::-1]],".")

In [ ]:
min_cost = min(cost)
plt.plot(hedging,[c/min_cost for c in cost[::-1]])
plt.plot(hedging,[c/min_cost for c in cost[::-1]],".")

In [ ]:
cost = []
hedging = np.arange(5,70,5)
init_state = 1e4
#hedging = np.arange(1,7)
j = 1
f,ax = plt.subplots(3,1,figsize=(16,8))
#plot_range = range(4000,5000)

duration = 6000
plot_range = range(0,6000)
#plot_range = range(0,300)
cumul =False
for h in reversed(hedging):
    thres = mu_drain+h*np.sqrt(mu_drain)
    #thres = h*10
    arrival_buffer,drain_buffer,zeta = simulate_reflected_random_walk(demand_seq[:duration],transfer_seq[:duration],drain_seq[:duration],thres,init_state=init_state)
    cost.append(sum(arrival_buffer*c1)+sum(drain_buffer*c2))
    #plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
    if cumul:
        ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label=h)
        ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range])
        ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
    else:
        ax[1].plot((drain_buffer*c2)[plot_range],label=h)
        ax[0].plot((arrival_buffer*c1)[plot_range])
        ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
    #print(np.min(np.diff((arrival_buffer[1500:2000]*c1+drain_buffer[1500:2000]*c2))))
    
thres = 1e6
#thres = h*10
arrival_buffer,drain_buffer,_ = simulate_reflected_random_walk(demand_seq[:duration],transfer_seq[:duration],drain_seq[:duration],thres,init_state=init_state)
#plt.plot(drain_buffer[j*1000:(j+1)*1000]*c2+arrival_buffer[j*1000:(j+1)*1000]*c1)
if cumul:
    #ax[1].plot(np.cumsum(drain_buffer*c2)[plot_range],label="e")
    ax[0].plot(np.cumsum(arrival_buffer*c1)[plot_range],label="e")
    #ax[2].plot(np.cumsum(arrival_buffer*c1+drain_buffer*c2)[plot_range])
else:
    #ax[1].plot((drain_buffer*c2)[plot_range],label="e")
    ax[0].plot((arrival_buffer*c1)[plot_range],label="e")
    #ax[2].plot((arrival_buffer*c1+drain_buffer*c2)[plot_range])
f.legend()

In [ ]:
(mu_transfer-mu_demand)/((zeta*mu_transfer)-mu_demand)

In [ ]:
min_cost = min(cost)
plt.plot(hedging,[c/min_cost for c in cost[::-1]])
plt.plot(hedging,[c/min_cost for c in cost[::-1]],".")

In [ ]:
min_cost = min(cost)
plt.plot(hedging,[c/min_cost for c in cost[::-1]])
plt.plot(hedging,[c/min_cost for c in cost[::-1]],".")

In [ ]:
h = []
for i in np.arange(0.94,0.949,0.001):
    h.append(1/(1-i))
    
plt.plot(np.arange(0.94,0.949,0.001)/0.94,[i/min(h) for i in h])

In [ ]:
min_cost = min(cost)

In [ ]:
cost[0]-cost[1]

In [ ]:
plt.plot(drain_buffer[:300])

In [ ]:
plt.plot(arrival_buffer[:600])

In [ ]:
plt.plot(buffer_seq[:1000])

In [ ]:
sum(buffer_seq)

In [ ]:
sum(buffer_seq)

In [ ]:
np.percentile((supply_seq-demand_seq)[(supply_seq-demand_seq) < 0],0.01)

In [ ]:
plt.plot(np.cumsum(supply_seq)-np.cumsum(demand_seq))

In [ ]:
percentile = 1.645
#percentile = 0
#percentile = 1.96
#percentile = 2.33
slack_capacity = mu_supply-mu_demand
time_horizon = (percentile**2)*mu_supply/(2*slack_capacity**2)
offset = time_horizon*(-slack_capacity) + percentile* np.sqrt(mu_supply*2*time_horizon)
print(time_horizon*2)
time_horizon = int(np.ceil(time_horizon))
offset = int(np.ceil(offset))

In [ ]:
time_horizon = (percentile**2)*mu_supply*2/slack_capacity**2
time_horizon = int(np.ceil(time_horizon))

In [ ]:
y = []
for d in range(time_horizon):
    y.append(d*(slack_capacity) - percentile* np.sqrt(mu_supply*2*d))

In [ ]:
y_1 = y
time_horizon_1 = time_horizon

In [ ]:
y_2 = y
time_horizon_2 = time_horizon

In [ ]:
time_horizon/time_horizon_1

In [ ]:
1.96/1.645

In [ ]:
plt.plot(range(time_horizon),y)
plt.plot(range(time_horizon_1),y_1)
plt.plot(range(time_horizon_2),y_2)

In [ ]:
y

In [ ]:
time_horizon

In [ ]:
offset

In [ ]:
thres = poisson.ppf(0.95,mu_demand)
#thres = 0
thres = poisson.ppf(0.5,mu_demand)

def idle_supply(demand_seq,supply_seq,offset):
    inv_pos = offset
    idle_supply_seq = np.zeros_like(supply_seq) 
    idle_count = 0
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        if inv_pos > thres+offset:
            s = 0
            idle_count += 1
        idle_supply_seq[i] = s
        inv_pos += s-d
    #print(idle_count/len(supply_seq)) 
    return idle_supply_seq

def idle_supply_time_horizon(demand_seq,supply_seq,offset,time_horizon):
    inv_pos = offset
    inv_pos_seq = np.zeros_like(supply_seq)
    days_count = 0
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        if (inv_pos > thres+offset) and days_count >= time_horizon:
            s = 0
            days_count = 0
        idle_supply_seq[i] = s
        inv_pos += s-d
        inv_pos_seq[i] = inv_pos
        days_count += 1
    return inv_pos_seq

def idle_supply_time_horizon_smooth(demand_seq,supply_seq,offset,time_horizon):
    inv_pos = offset
    inv_pos_seq = np.zeros_like(supply_seq)
    days_count = 0
    just_idled = False
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        surplus = inv_pos - offset
        if surplus > 0 and ((days_count >= time_horizon) or just_idled):
            if d > surplus:
                s = d-surplus
            else:
                s = 0
            days_count=0
            just_idled = True
        else:
            just_idled = False
            
        inv_pos += s-d
        inv_pos_seq[i] = inv_pos
        if not just_idled:
            days_count += 1
            
    return inv_pos_seq

def work_supply_time_horizon_smooth(demand_seq,supply_seq,offset,time_horizon):
    inv_pos = offset
    inv_pos_seq = np.zeros_like(supply_seq)
    days_count = 0
    just_idled = True
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        surplus = inv_pos - offset
        if surplus > 0 and ((days_count >= time_horizon) or just_idled):
            days_count = 0
            if d > surplus:
                s = d-surplus
            else:
                s = 0
            days_count=0
            just_idled = True
        else:
            days_count += 1
            just_idled = False
            
        inv_pos += s-d
        inv_pos_seq[i] = inv_pos
            
    return inv_pos_seq

def idle_supply_smooth(demand_seq,supply_seq,offset):
    inv_pos = offset
    idle_supply_seq = np.zeros_like(supply_seq) 
    idle_count = 0
    inv_pos_array = np.zeros_like(supply_seq)
    for i,(d,s) in enumerate(zip(demand_seq,supply_seq)):
        surplus = inv_pos - offset
        if surplus > 0:
            if d > surplus:
                s = d-surplus
            else:
                s = 0
            idle_count += 1
            
        idle_supply_seq[i] = s
        inv_pos += s-d
        inv_pos = min(inv_pos,offset)
        inv_pos_array[i] = inv_pos

    #print(idle_count/len(supply_seq)) 
    print(inv_pos)
    return inv_pos_array

In [ ]:
slack_capacity/np.sqrt(2*mu_demand)

In [ ]:
point = 1400
plt.plot(inv_pos_seq[point-100:point+500])

In [ ]:
point = 1400
plt.plot(inv_pos_seq[point-100:point+500])

In [ ]:
point = 1400
plt.plot(inv_pos_seq[point-100:point+100])

In [ ]:
offset

In [ ]:
time_horizon*slack_capacity/2

In [ ]:
slack_capacity

In [ ]:
inv_pos_seq = work_supply_time_horizon_smooth(demand_seq,supply_seq,53,12)

print(np.mean(inv_pos_seq < 0))

In [ ]:
inv_pos_seq = idle_supply_time_horizon_smooth(demand_seq,supply_seq,53,12)

print(np.mean(inv_pos_seq < 0))

In [ ]:
stocks = inv_pos_seq.copy()
stocks[inv_pos_seq < 0] = 0
np.mean(stocks)

In [ ]:
inv_pos_seq = idle_supply_time_horizon_smooth(demand_seq,supply_seq,41,69)

print(np.mean(inv_pos_seq < 0))

In [ ]:
stocks = inv_pos_seq.copy()
stocks[inv_pos_seq < 0] = 0
np.mean(stocks)

In [ ]:
inv_pos_seq = idle_supply_time_horizon(demand_seq,supply_seq,offset,time_horizon)

print(np.mean(inv_pos_seq < 0))
#plt.plot(inv_pos_seq[827341-10:827341+10])
#plt.plot(inv_pos_seq[827341-10:827341+10],".")

In [ ]:
stocks = inv_pos_seq.copy()
stocks[inv_pos_seq < 0] = 0
np.mean(stocks)

In [ ]:
idle_supply_seq,inv_pos_seq = idle_supply_smooth(demand_seq,supply_seq, np.ceil(offset))
#inv_pos_seq = offset + np.cumsum(idle_supply_seq)-np.cumsum(demand_seq)
print(np.mean(inv_pos_seq < 0))
#plt.plot(inv_pos_seq[827341-10:827341+10])
#plt.plot(inv_pos_seq[827341-10:827341+10],".")

plt.plot(inv_pos_seq[:1200])

In [ ]:
n_sims = 100000
demand_sum = np.random.poisson(mu_demand*np.ceil(time_horizon),n_sims)
supply_sum = np.random.poisson(mu_supply*np.ceil(time_horizon),n_sims)

print(np.mean((demand_sum-supply_sum) > np.ceil(offset)))

In [ ]:
offset+time_horizon*slack_capacity

In [ ]:
1001 % 100

In [ ]:
offset

In [ ]:
time_horizon*slack_capacity/2

In [ ]:
np.random.seed(500)
n_sims = 100000
#n_sims = 20
stockouts = []
last_day_stockouts = []
last_day_stockouts_vals = []
ave_inventories = []
sim_time_horizon = time_horizon
for i in range(n_sims):
    demand = np.random.poisson(mu_demand,sim_time_horizon)
    supply = np.random.poisson(mu_supply,sim_time_horizon)
    inv_pos_seq = offset + np.cumsum(supply)-np.cumsum(demand)
    stockouts.append(np.sum(inv_pos_seq < 0))
    last_day_stockouts.append(inv_pos_seq[-1] < offset)
    if last_day_stockouts[-1]:
        last_day_stockouts_vals.append(inv_pos_seq[-1]-offset)
    
    ave_inventories.append(np.mean(inv_pos_seq))
    if i % 10000 == 0:
        plt.plot(inv_pos_seq)
    
sum(stockouts)/(sim_time_horizon*n_sims),np.sum(last_day_stockouts)/(n_sims),np.mean(ave_inventories)


In [ ]:
offset

In [ ]:
np.median(last_day_stockouts_vals)

In [ ]:
for offset in range(200):
    stock_out_probs = []
    for d in range(1,time_horizon+1):
        stock_out_prob = norm.cdf(-offset,slack_capacity*d,np.sqrt(2*mu_supply*d))
        stock_out_probs.append(stock_out_prob)
    overal_stockout_prob = np.mean(stock_out_probs)
    #print(overal_stockout_prob)
    if overal_stockout_prob < 0.05:
        break

In [ ]:
time_horizon

In [ ]:
def get_percentile_deficit(cycle_dur,slack_capacity,variance,percentile = 0.5):
    mu = slack_capacity*cycle_dur
    std = np.sqrt(variance*cycle_dur)
    cum_deficit_prob = norm.cdf(0,mu,std)
    cum_percentile = 0
    prev_cum_prob = cum_deficit_prob
    for i in range(10000):
        cum_prob = norm.cdf(-i,mu,std)
        prob = (prev_cum_prob - cum_prob)/cum_deficit_prob
        cum_percentile += prob
        if cum_percentile >= percentile:
            return i
        prev_cum_prob = cum_prob
        
a = get_percentile_deficit(time_horizon/4,slack_capacity,2*mu_supply)
#get_percentile_deficit(slack_capacity,2*mu_supply,time_horizon)
print(a)

def compute_recovery_time(slack_capacity,variance,deficit,bound = 2.33):
    dur = ((bound*np.sqrt(variance)+np.sqrt(bound**2*variance+4*slack_capacity*deficit))/(2*slack_capacity))**2
    return int(np.ceil(dur))

print(compute_recovery_time(slack_capacity,2*mu_supply,a))

def get_average_stockout_prob(duration,slack_capacity,variance,start):
    stock_out_probs = []
    for d in range(1,duration+1):
        stock_out_prob = norm.cdf(0,start+slack_capacity*d,np.sqrt(variance*d))
        stock_out_probs.append(stock_out_prob)
    average_stockout_prob = np.mean(stock_out_probs)
    return average_stockout_prob

def compute_stockout_prob_and_inventory_cost(cycle_dur,slack_capacity,variance,offset):
    mu = slack_capacity*cycle_dur
    std = np.sqrt(variance*cycle_dur)
    cum_deficit_prob = norm.cdf(0,mu,std)
    #print(cum_deficit_prob)
    deficit = get_percentile_deficit(cycle_dur,slack_capacity,variance,0.95)
    #print(deficit)
    rec_dur = compute_recovery_time(slack_capacity,variance,deficit)
    #print(rec_dur)
    cycle_stockout_prob = get_average_stockout_prob(cycle_dur,slack_capacity,variance,offset)
    
    rec_dur = int(np.ceil(deficit/slack_capacity))
    print(rec_dur)
    rec_stockout_prob = get_average_stockout_prob(rec_dur,slack_capacity,variance,offset-deficit)
    #print(cycle_stockout_prob,rec_stockout_prob)
    
    effective_duration = (cycle_dur+cum_deficit_prob*rec_dur)
    #print(cycle_dur/effective_duration)
    
    overall_stockout_prob = (cycle_dur*cycle_stockout_prob+cum_deficit_prob*rec_dur*rec_stockout_prob)/effective_duration
    
    overall_inventory_cost = (cycle_dur*(0.5*slack_capacity*cycle_dur+offset)+cum_deficit_prob*rec_dur*(0.5*slack_capacity*rec_dur+offset-deficit))/effective_duration
    #print(overall_inventory_cost)
    return overall_stockout_prob,overall_inventory_cost

In [ ]:
time_horizon/4

In [ ]:
variance = 2*mu_supply

In [ ]:
min_inv_cost = np.inf
min_cycle_dur = None
min_offset = None
for cycle_dur in range(1,int(time_horizon)):

    for offset in range(200):
        overall_stockout_prob,inv_cost = compute_stockout_prob_and_inventory_cost(cycle_dur,slack_capacity,variance,offset)
        #print(overall_stockout_prob)
        if overall_stockout_prob < 0.05:
            break
    
    print(cycle_dur,inv_cost)
    if inv_cost < min_inv_cost:
        print(cycle_dur)
        min_inv_cost = inv_cost
        min_cycle_dur = cycle_dur
        min_offset = offset
    
print(offset)

In [ ]:
min_offset

In [ ]:
min_cycle_dur

In [ ]:
min_inv_cost

In [ ]:
time_horizon

In [ ]:
int(time_horizon)*(0.5*slack_capacity)

In [ ]:
inv_cost

In [ ]:
print(overal_stockout_prob)

In [ ]:
overal_stockout_prob

In [ ]:
probs = []
deficit = 10000
for i in range(deficit):
    v = -offset-i
    mu = slack_capacity*time_horizon
    std = np.sqrt(2*mu_supply*time_horizon)
    probs.append(norm.cdf(v,mu,std))
    #print(i,probs[-1])
    
np.sum(-np.diff(probs)*np.arange(1,deficit)/norm.cdf(-offset,mu,std))

In [ ]:
offsets = []
for dur in range(1,time_horizon+1):
    for offset in range(200):
        stock_out_probs = []
        for d in range(1,dur+1):
            stock_out_prob = norm.cdf(-offset,slack_capacity*d,np.sqrt(2*mu_supply*d))
            stock_out_probs.append(stock_out_prob)
        overal_stockout_prob = np.mean(stock_out_probs)
        #print(overal_stockout_prob)
        if overal_stockout_prob < 0.05:
            break
    #print(dur,offset)
    offsets.append(offset)

In [ ]:
plt.plot(offsets)

In [ ]:
norm.cdf(-offset,mu,std)

In [ ]:
offset

In [ ]:
mu

In [ ]:
(-np.diff(probs)/norm.cdf(-offset,mu,std))[:50]

In [ ]:
-np.diff(probs)/norm.cdf(-offset,mu,std)

In [ ]:
offset

In [ ]:
np.sum(last_day_stockouts)/(n_sims)

In [ ]:
sum(stockouts)/(int(np.ceil(time_horizon))*n_sims)

In [ ]:
np.sum(last_day_stockouts)

In [ ]:
np.sum(last_day_stockouts)/sum(stockouts)

In [ ]:
np.mean(stockouts)

In [ ]:
stockouts = np.array(stockouts)

In [ ]:
np.median(stockouts[stockouts > 0])

In [ ]:
plt.hist(stockouts[stockouts > 0])

In [ ]:
plt.hist(stockouts,bins=range(0,50,2))

In [ ]:
2*time_horizon

In [ ]:
norm.cdf(-offset,slack_capacity*10,np.sqrt(mu_supply*10))

In [ ]:
int(np.ceil(time_horizon))